# December 15, 2022
https://adventofcode.com/2022/day/15

In [ ]:
# too high: 5461730

In [1]:
import re

fn = "../data/2022/15.txt"
sensors = []
beacons = []

with open(fn, "r") as file:
    while True:
        line = file.readline()
        if not line:
            break

        match = re.search(".*x=(-?\d+).*y=(-?\d+).*x=(-?\d+).*y=(-?\d+)", line)
        sensors.append( [int(match.group(1)), int(match.group(2))] )
        beacons.append( [int(match.group(3)), int(match.group(4))] )

In [2]:
def manhattan( a, b ):
    return abs(a[0] - b[0]) + abs(a[1] - b[1])

def distance( a, b, type="manhattan" ):
    if type == "manhattan":
        return manhattan(a,b)
    else:
        raise Exception(f'''Distance type {type} not known''')

In [102]:
def reduce_borel_set( borel_set ):
    i = 0
    # use while loops since set length will change
    while i < len(borel_set):
        j = i+1
        while j < len(borel_set):
            un = union( borel_set[i], borel_set[j] )
            if un:
                # combine set j into set i
                borel_set[i] = un
                # drop set j
                borel_set = borel_set[:j] + borel_set[j+1:]
                # but now need to check for earlier overlaps!
                j = i+1
            else:
                # nothing to do, keep going
                j += 1
        i += 1
    borel_set.sort()
    return borel_set

def borel_size( borel_set ):
    # for this problem, both ends are inclusive
    return sum( x[1] - x[0] + 1 for x in borel_set )



def do_intersect( set1, set2 ):
    # two intervals overlap if one of their mins is within the other interval
    return (
               (set1[0] >= set2[0] and set1[0] <= set2[1]) or
               (set2[0] >= set1[0] and set2[0] <= set1[1])
    )

def union( set1, set2 ):
    if do_intersect( set1, set2 ):
        return [min(set1[0], set2[0]), max(set1[1], set2[1])]
    else:
        return False



In [103]:
reduce_borel_set( [[0,10], [5,15], [20, 100], [1000,2000], [13,50]] )

[[0, 100], [1000, 2000]]

### Part 1

In [104]:
def scan_row( sensors, beacons, row = int(2e6), verbose=False ):
    elim_set = []
    for i in range(len(sensors)):
        s = sensors[i]
        b = beacons[i]

        dist = distance(s,b)
        ht = distance( s, [s[0], row] )
        if verbose:
            print(i)
            print("SENSOR:", *s)
            print("BEACON:", *b)
            print(dist, ht)

        if ht <= dist:
            elim = [ s[0] - (dist-ht), s[0] + (dist-ht) ] 
            elim_set.append(elim)
            if verbose:
                print(elim)
                print("\n")

    return reduce_borel_set( elim_set )

In [110]:
vref = 2000000
elim = scan_row( sensors, beacons, row=vref )
print(elim)

count = borel_size( elim )
# exclude spaces with beacons:
count -= len(list(set( [x[0] for x in beacons if x[1]==vref] )))


print(f'''No beacons in {count} spaces.''')

[[-774791, 4686938]]
No beacons in 5461729 spaces.


### Part 2
This solution is not efficient because we handle each row independently, but it's a reasonable wait

In [112]:
minx = 0
miny = 0
maxx = 4000000
maxy = 4000000

pos = None
for vref in range(miny, maxy+1):
    if vref % 100000 == 0:
        print(vref)
    elim_set = scan_row( sensors, beacons, row=vref )
    elim_set = reduce_borel_set( elim_set )

    # check to left:
    if elim_set[0][0] > minx:
        pos = [0, vref]
    # check to right:
    elif elim_set[-1][1] < maxx:
        pos = [maxx, vref]
    # no holes to check, continue to next row
    elif len(elim_set) == 1:
        continue
    # else check wholes in elim_set
    else:
        for i in range(len(elim_set)-1):
            set1 = elim_set[i]
            set2 = elim_set[i+1]

            # there's a space to the right of first set
            if set1[1] < maxx:
                pos = [set1[1]+1, vref]
            # there's a space to left of second set
            elif set2[0] > minx: 
                pos = [set2[0]-1, vref]
                break
    if pos is not None:
        break

0
100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000
1800000
1900000
2000000
2100000
2200000
2300000
2400000
2500000
2600000
2700000
2800000
2900000
3000000
3100000


In [113]:
pos

[2655411, 3166538]

In [115]:
pos[0]*4000000 + pos[1]

10621647166538